# RSI
## Overview
RSI is a indicator used by many. It was created in 1978 by J. Welles Wilder Jr. in his New Concepts in Technical Trading Systems. It attempts to normalize a tending direction of an underlying asset. It first determines the RS (Relative Strength) of the moving prices by finding the avg gain over the avg loss. 

In [2]:
import pandas as pd
import numpy as np
import warnings

def mean_avg_fun(inputs: np.ndarray) -> float:
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", category=RuntimeWarning)
        return inputs.mean()

def RSI(prices: np.ndarray, avg_func = mean_avg_fun):
    delta = np.diff(prices)
    gains = delta[delta > 0]
    losses = -delta[delta < 0]

    avg_gain = avg_func(gains)
    avg_loss  = avg_func(losses)

    if avg_loss == 0:
        return 100
    if avg_gain == 0:
        return 0
    
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))

    return rsi

In [3]:
spx_data = pd.read_csv(
    './data/SPX_2019_1min.csv',
    header=None,
    names=["date", "open", "high", "low", "close"],
    parse_dates=["date"],
    index_col=['date']
)

In [11]:

def buy_sell_RSI(data: pd.DataFrame, period, buy_low, sell_high):
    c_data = data.copy()
    c_data["RSI"] = c_data['close'].rolling(period).apply(lambda close: RSI(close), raw=True)
    
    in_active_position = False
    buy_price = 0
    trade_pl = 0
    
    for index, row in c_data[(c_data["RSI"] > sell_high + 1) | (c_data["RSI"] < buy_low + 1)].iterrows():
        if not in_active_position:
            if row["RSI"] < buy_low:
                buy_price = row["close"]
                in_active_position = True
        else:
            if row["RSI"] > sell_high:
                trade_pl = trade_pl + (row["close"] - buy_price)
                buy_price = 0
                in_active_position = False

    return trade_pl

In [12]:
pls = {}
for period in range(3, len(spx_data)):
    for buy_low in range(1, 50):
        for sell_high in range(51, 100):
            pl = buy_sell_RSI(spx_data, period, buy_low, sell_high)
            pls[f"{period},{buy_low},{sell_high}"] = pl
            print(f"{period},{buy_low},{sell_high} = {pl}")

3,1,51 = -19.969999999998436
3,1,52 = -22.009999999997945
3,1,53 = -19.60999999999831
3,1,54 = -19.199999999998
3,1,55 = -22.56999999999789
3,1,56 = -19.539999999998145
3,1,57 = -19.539999999998145
3,1,58 = -17.089999999997872
3,1,59 = -11.419999999998254
3,1,60 = -10.969999999998436
3,1,61 = -9.549999999998363
3,1,62 = -8.919999999998709
3,1,63 = -10.059999999999036
3,1,64 = -10.639999999998963
3,1,65 = -10.759999999998854
3,1,66 = -9.459999999999127
3,1,67 = -9.389999999998963
3,1,68 = -5.679999999998927
3,1,69 = -16.37999999999829
3,1,70 = -11.84999999999809
3,1,71 = -12.059999999998126
3,1,72 = -16.10999999999831
3,1,73 = -4.6599999999980355
3,1,74 = -13.029999999998381
3,1,75 = -17.479999999998654
3,1,76 = -8.449999999998909
3,1,77 = -9.449999999998909
3,1,78 = -21.489999999999327
3,1,79 = -22.999999999999545
3,1,80 = -18.840000000000146
3,1,81 = -8.220000000000255
3,1,82 = 5.779999999999745
3,1,83 = 23.629999999999654
3,1,84 = 21.989999999999327
3,1,85 = 14.9399999999996
3,1,86 =

KeyboardInterrupt: 

In [ ]:
print(f"Optimal parameters: {pls}")

In [ ]:
pl = buy_sell_RSI(spx_data.first("6M"), 7, 31, 92)
pl2 = buy_sell_RSI(spx_data.first("6M"), 14, 30, 70)

print(pl)
print(pl2)